In [23]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
!pip install pandas


  Using cached pandas-1.1.4-cp37-cp37m-manylinux1_x86_64.whl (9.5 MB)
  Using cached pytz-2020.4-py2.py3-none-any.whl (509 kB)


In [4]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [1]:
!pip install scipy

     |████████████████████████████████| 25.9 MB 1.1 MB/s eta 0:00:01


In [2]:
!pip install sklearn

     |████████████████████████████████| 6.8 MB 2.1 MB/s eta 0:00:01     |█████████▋                      | 2.1 MB 2.1 MB/s eta 0:00:03
     |████████████████████████████████| 301 kB 27.9 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=856168b32c9dafb710059ceb20bfb250c78b30507cbe60477ed21e58a85d6fa4
  Stored in directory: /home/jovyan/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn



# In this challenge,
I  created a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [5]:
# import csv data into dataframes
df_books = pd.read_csv(
    'BX-Books.csv',
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    'BX-Book-Ratings.csv',
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [6]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [7]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [8]:
num_user_ratings = df_ratings.groupby(['user']).count()
num_user_ratings.drop(columns=['isbn'], inplace=True)
num_user_ratings.head()

,rating
user,
2,1
7,1
8,18
9,3
10,2


In [9]:
num_book_ratings = df_ratings.groupby(['isbn']).count()
num_book_ratings.drop(columns=['user'], inplace=True)
num_book_ratings.head()

,rating
isbn,
0330299891,2
0375404120,2
0586045007,1
9022906116,2
9032803328,1


In [11]:
users_to_omit = list(num_user_ratings.loc[num_user_ratings['rating'] < 200].index.values)
books_to_omit = list(num_book_ratings.loc[num_book_ratings['rating'] < 100].index.values)

In [12]:
# Remove books with less than 100 ratings and users with less than 200 ratings
indices_to_drop_users = list(df_ratings.loc[df_ratings['user'].isin(users_to_omit)].index.values)
indices_to_drop_books = list(df_ratings.loc[df_ratings['isbn'].isin(books_to_omit)].index.values)
indices_to_drop = list(set(indices_to_drop_users).union(set(indices_to_drop_books)))
df_ratings.drop(index=indices_to_drop, inplace=True)

indices_to_drop_df_books = list(df_books.loc[df_books['isbn'].isin(books_to_omit)].index.values)
df_books.drop(index=indices_to_drop_df_books, inplace=True)

In [13]:
# pivot ratings into matrix style
df_ratings_matrix = df_ratings.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)

# convert dataframe to scipy sparse matrix
mat_ratings_matrix = csr_matrix(df_ratings_matrix.values)

# mapping from index in sparse matrix to isbn
index_hashmap = list(df_ratings_matrix.index.values)

In [14]:
print(mat_ratings_matrix.shape)
df_ratings_matrix.head()

(731, 888)


user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
neighbours = NearestNeighbors(n_neighbors=6, algorithm="auto")
neighbours.fit(mat_ratings_matrix)

NearestNeighbors(n_neighbors=6)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

In [21]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    bookData = df_books.loc[df_books['title'] == book]
    if len(bookData) != 1:  # book is not in database
        return [book, ["", 0.]*5]

    isbn  = bookData.iloc[0,:]['isbn']
    index = index_hashmap.index(isbn)
    neigh_dist, neigh_ind = neighbours.kneighbors(X=mat_ratings_matrix[index, :], n_neighbors=6)

    recommended_books = [book, []]
    for ind, dist in zip(neigh_ind[0][1:], neigh_dist[0][1:]):
        isbn = index_hashmap[ind]
        title = df_books.loc[df_books['isbn'] == isbn].iloc[0, :]['title']
        recommended_books[1].append([title, dist])

    return recommended_books

Use the cell below to test your function. The test_book_recommendation() function will inform you if you passed the challenge or need to keep trying.

In [22]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False
    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
    for i in range(2): 
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Surgeon', 61.286213], ['Unspeakable', 61.522354], ['The Perks of Being a Wallflower', 61.579216], ['Gap Creek: The Story Of A Marriage', 61.676575], ['The Weight of Water', 61.75759]]]
You havn't passed yet. Keep trying!
